<a href="https://colab.research.google.com/github/asetya/BigData/blob/master/installsparkStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PERSIAPAN**

1. Install Java open Jdk 

In [1]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

2. Download spark 


In [2]:
# Install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

3. Unzip spark

In [8]:
# Unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

4. Setting Java Home dan Spark Home

In [35]:
# Set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

6. Install findspark 

In [4]:
# Install findspark using pip
!pip install -q findspark

7. Inisialisasi Spark

In [48]:
import findspark
findspark.init() 

8. Buat Fungsi khusus untuk parsing data dari sensor 

In [49]:
# Parse a line of weather station data, returning the average wind direction measurement 
#
import re
def parse(line):
    match = re.search("Dm=(\d+)", line)
    if match:
        val = match.group(1)
        return [int(val)]
    return []

**RUN THE STREAMING**

1. Import and create streaming context. Next, we will import and create a new instance of Spark's StreamingContext:

In [77]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
#create spark context
sc = SparkContext("local[2]", "NetworkWordCount")
#membuat streaming context dengan batch interval = 1 detik 
ssc = StreamingContext(sc, 1)



2. Membuat koneksi stream dengan data weather pada alamat rtd.hpwren.ucsd.edu port 12028

In [82]:
#membaca baris demi baris pada rtd.hpwren.ucsd.edu socket 12028 
lines = ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12020)
# Read measurement. Next, let's read the average wind speed from each line and store it in a new DStream vals:
vals = lines.flatMap(parse)


Membuat sliding window  data 10 detik dan bergeser setiap 5 detik

```
# This is formatted as code
```



In [83]:
window = vals.window(10,5)

Mebuat fungsi untuk melakukan analisis - misalnya kita bikin fungsi untuk mencari nilai minimium dan maksimum dalam sebuiahg winwdws 

In [84]:
def stats(rdd): 
  print(rdd.collect())
  if rdd.count()>0:
    print ("max ={},min={}".format(rdd.max(), rdd.min()))
  


memanggil fungsi stats() untuk setiap RDD [ada sliding window kita

In [85]:
window.foreachRDD(lambda rdd:stats(rdd))



2. menjalankan streaming 

In [86]:
#@title
ssc.start()

3. Mengakhiri streaming

In [87]:
ssc.stop()

[104, 104, 104, 104]
max =104,min=104
[15, 15, 15, 15, 15]
max =15,min=15
[104, 104, 104, 104, 104, 103, 103, 103, 104]
max =104,min=103
[15, 15, 15, 15, 15, 15, 15, 15, 15, 16]
